In [10]:
!pip install flask

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import pandas as pd

dataCSV = pd.read_csv('fraudtestdumed.csv')
columnas = dataCSV.columns.tolist()

In [12]:
columnas

['cc_num',
 'amt',
 'is_fraud',
 'Time',
 'merchant_fraud_Abbott-Rogahn',
 'merchant_fraud_Abbott-Steuber',
 'merchant_fraud_Abernathy and Sons',
 'merchant_fraud_Abshire PLC',
 'merchant_fraud_Adams, Kovacek and Kuhlman',
 'merchant_fraud_Adams-Barrows',
 'merchant_fraud_Altenwerth, Cartwright and Koss',
 'merchant_fraud_Altenwerth-Kilback',
 'merchant_fraud_Ankunding LLC',
 'merchant_fraud_Ankunding-Carroll',
 'merchant_fraud_Armstrong, Walter and Gottlieb',
 'merchant_fraud_Auer LLC',
 'merchant_fraud_Auer-Mosciski',
 'merchant_fraud_Auer-West',
 'merchant_fraud_Bahringer Group',
 'merchant_fraud_Bahringer, Bergnaum and Quitzon',
 'merchant_fraud_Bahringer, Osinski and Block',
 'merchant_fraud_Bahringer, Schoen and Corkery',
 'merchant_fraud_Bahringer-Larson',
 'merchant_fraud_Bahringer-Streich',
 'merchant_fraud_Bailey-Morar',
 'merchant_fraud_Balistreri-Nader',
 'merchant_fraud_Barrows PLC',
 'merchant_fraud_Bartoletti and Sons',
 'merchant_fraud_Bartoletti-Wunsch',
 'merchant_fra

In [13]:
from flask import Flask, request, jsonify
import sqlite3
from sqlite3 import Error
import pandas as pd
import os

dataCSV = pd.read_csv('fraudtestdumed.csv')
columnas = dataCSV.columns.tolist()

def create_connection():
    dir_path = os.path.dirname(os.path.realpath(__file__))
    db_file = os.path.join(dir_path, "bank.db")

    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
        raise e 

    return conn

def create_table(conn, columnas):
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='creditcard'")
    if cur.fetchone() is None:
        try:
            sql = '''CREATE TABLE creditcard({})'''.format(', '.join(columnas))
            conn.execute(sql)
            conn.commit()
            print("Tabla 'creditcard' creada exitosamente.")
        except Error as e:
            print("Error al crear la tabla 'creditcard':", e)

def insert_datos(conn, info):
    try:
        cur = conn.cursor()
        columns = ', '.join(info.keys())
        placeholders = ', '.join('?' * len(info))
        sql = f"INSERT INTO creditcard ({columns}) VALUES ({placeholders})"
        cur.execute(sql, tuple(info.values()))
        conn.commit()
        print("Datos insertados exitosamente en la tabla 'creditcard'.")
    except Error as e:
        print("Error al insertar datos en la tabla 'creditcard':", e)

def check_database(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM creditcard")

    rows = cur.fetchall()

    for row in rows:
        print(row)


app = Flask(__name__)

@app.route('/datos', methods=['POST'])
def post_datos():
    info = request.get_json()
    conn = create_connection()
    create_table(conn,dataCSV.columns.tolist())
    insert_datos(conn, info)
    check_database(conn)
    return jsonify({'message': '¡Información guardada con éxito!'}), 201

if __name__ == '__main__':
    app.run(debug=True, port=5050)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/byshadowoz/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/byshadowoz/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/byshadowoz/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/byshadowoz/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/byshadowoz/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in ini

SystemExit: 1

/home/byshadowoz/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
